# Chamando Funções com o Google Gemini (Introdução?)

Já vimos aqui no blog como usar o **Google Gemini** (e outros modelos de linguagem) para gerar texto.
Vimos, também, várias técnicas de **engenharia de prompts** para melhorar a qualidade ou definir o formato da saída produzida.

Porém, essas técnicas  esbarram em algumas limitações dos LLMs:
- eles só conhecem até certa data limite (dos dados usados no treinamento)
- não sabem sobre agendamentos futuros (e.g. jogos de futebol que vão acontecer)
- não são confiáveis em cálculos matemáticos

Para superar várias dessas limitações, podemos permitir que o LLM use **chamadas de função** para obter novas informações.

Vamos conhecer esse recurso, na prática, neste post! Veremos como fornecer ao Google Gemini uma função que multiplica dois números.


## 1 - Configurações Iniciais

Você vai precisar gerar uma chave da API GOOGLE, salvar com o nome GOOGLE_API_KEY em um arquivo `.env` e carregar este arquivo com o módulo `dotenv`. 

Qualquer dúvida, ver *postagem anterior* (onde teria sido ensinado como usar o básico do Google Gemini).

In [1]:
from dotenv import load_dotenv, find_dotenv
import google.generativeai as genai

_ = load_dotenv(find_dotenv()) # carrega a variável GOOGLE_API_KEY, que é importada automaticamente pelo módulo google.generativeai

c:\Users\pablo\.conda\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# não precisa executar esse código, pois a variável GOOGLE_API_KEY é carregada automaticamente no módulo google.generativeai
#import os
#genai.configure(api_key=os.getenv('GOOGLE_API_KEY')


Você quer ver os nomes dos modelos disponíveis? Execute o código abaixo.

Vamos focar o restante deste artigo no `models/gemini-1.5-flash`, que é o principal modelo de linguagem do Google (na data de escrita deste blog).

In [3]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


## 2 - Exemplo: Função para multiplicar

As primeiras versões do ChatGPT eram conhecidas por suas falhas eventuais com operações matemáticas. Os modelos mais recentes melhoraram um pouco, mas ainda estão sujeitos a esses erros.

Vamos ver como usar **chamada de função** para fazer o Gemini não errar mais as "continhas de multiplicação"?

Primeiro, vamos definir uma função que multiplica dois números. 

### Definindo a função

In [4]:
def multiply(a:float, b:float) -> float:
    """Use esta função sempre que precisar multiplicar dois números 'a' e 'b', 
    seja por requisição direta do usuário ou para resolver algum problema matemático que envolva multiplicação."""
    print("mult", a, b)
    return a*b

Note que eu defini os tipos de parâmetros e o tipo de retono. 
Você também deve ter percebido que fiz uma documentação com um texto bem detalhado. 

Tudo é isso é muito importante, porque toda essa documentação vai ser enviada para o Gemini como parte do prompt (a biblioteca faz isso automaticamente). 

No nosso exemplo, fiz algumas mudanças até chegar à documentação acima. A documentação dada consegue para "forçar" o modelo a usar a função em (quase) todas as situações desejadas.
Enfim, isso mostra que definir a documentação é um tipo especial de *engenharia de prompts* também! 

### Fornecendo a função ao modelo

E agora, como faço para o Gemini chamar essa função? Bem, pasta usar o parâmetro `tools` (trad.: "ferramentas") no construtor da classe `GenerativeModel`, que usamos para acessar o modelo.

O parâmetro `tools` recebe uma lista de funções. Como só temos uma função ser a lista unitária `[multiply]` contendo apenas a função acima.

In [5]:
model1 = genai.GenerativeModel(model_name='gemini-1.5-flash', tools=[multiply])

Uma vantagem do Gemini (em relação aos modelos da OpenAI), no momento, é que a biblioteca do Google permite fazer a chamada à função automaticamente. Para isso, basta fazer a configuração abaixo.

Mas essa chamada, na verdade, é feita localmente. Por isso, vamos ver localmente o resultado do "print" inserido no código da função.

In [6]:
chat1 = model1.start_chat(enable_automatic_function_calling=True)

- Criar um **diagrama** mostrando a dinâmica: código cliente local -> modelo Gemini remoto -> dispara função local

### Um teste simples

Agora, vamos chamar o modelo, passando um prompt que referencia alguma conta de multiplicação.

In [7]:
response1 = chat1.send_message("Quanto é 293874298 multiplicado por 7 milhões?")
print(response1.text)

mult 293874298.0 7000000.0
2.057120086e+15


### Um teste mais complexo

E se fizermos uma referência mais indireta à multiplicação?

Vamos pedir ao Gemini para resolver um probleminha básico envolvendo esta operação, para ver como ele se sai.

Desta vez, vamos exibir o resultado como texto *markdown*, para execução em uma célula de um notebook Jupyter.

In [8]:
from IPython.display import Markdown

response2 = chat1.send_message("Tenho quatro mil e sete galpões, cada um com 1731 galinhas.\n" 
                               "Para cada galinha, preciso de 7.32 litro de água por dia.\n" 
                               "De quantos litros de água eu preciso por dia?")
Markdown(response2.text)

mult 4007.0 1731.0
mult 6936117.0 7.32


Você precisa de 50.772.376,44 litros de água por dia. 


Por ser mais sutil, este é um caso que, às vezes, pode não funcionar como esperado!

Mas sabemos que funcionou porque vemos duas mensagens (de `print`) disparadas dentro da função `multiply`. Note que elas correspondem às multiplicações necessárias para resolver o problema!

## 3 - Quando a Função não é Chamada

Normalmente, o modelo pode decidir não chamar a função. Vamos destacar três casos em que ele não chama.

***1o caso***: Se sua mensagem nada tiver a ver com multiplicação, ele não vai chamar!

No exemplo abaixo, continuamos o chat anterior criando uma pergunta que não precisa de multiplicação para ser respondida.

In [9]:
response3 = chat1.send_message("O que é um número primo? Dê uma resposta curta.")
Markdown(response3.text)

Um número primo é um número inteiro maior que 1 que é divisível apenas por 1 e por ele mesmo. 


***2o caso***: Se, no mesmo chat, ele precisar do mesmo resultado duas vezes (por exemplo, duas multiplicações idênticas entre os mesmos dois números), ele *pode* optar por não chamar a função novamente!

Isso acontece porque o resultado fica registrado no histórico e, assim, o modelo pode simplesmente reusá-lo. (Vamos mostrar o histórico mais adiante.)

Mas não é muito previsível. Às vezes, o modelo chama a função desnecessariamente também. 

No exemplo abaixo, tive que pedir explicitamente para ele resar um resultado anterior.

In [10]:
response4 = chat1.send_message("Quanto é mesmo 4007.0 vezes 1731.0? Reuse o resultado anterior.")
Markdown(response4.text)

O resultado anterior já nos mostrou que 4007.0 vezes 1731.0 é 6.936.117. 


***3o caso***: Também pode acontecer de ele não chamar a função (em uma situação em que deveria chamar) por mera *falha* do modelo. 

Para dar um exemplo:
- vamos criar um novo modelo  (`model2`), passando uma função de multiplicação com uma documentação mais simples 
- vamos criar um novo chat (`chat2`) com este modelo
- depois, vamos pedir novamente para o modelo resolver o probleminha matemático anterior.

In [11]:
def multiply2(a:float, b:float) -> float:
    """Multiply numbers."""
    print("mult", a, b)
    return a*b

model2 = genai.GenerativeModel(model_name='gemini-1.5-pro', tools=[multiply2])
chat2 = model2.start_chat(enable_automatic_function_calling=True)
response5 = chat2.send_message("Tenho quatro mil e sete galpões, cada um com 1731 galinhas.\n" 
                               "Para cada galinha, preciso de 7.32 litro de água por dia.\n" 
                               "De quantos litros de água eu preciso por dia?")
Markdown(response5.text)

Você tem um total de 4007 galpões * 1731 galinhas/galpão = 6935117 galinhas.

Portanto, você precisa de 6935117 galinhas * 7.32 litros/galinha = 50820716.44 litros de água por dia. 


O modelo não fez as chamadas de função necessárias... também não acertou as contas sozinho... Assim, a resposta final está errada!

Como já comentamos antes, para evitar essa falha, vale a pena tentar melhorar a documentação da função. Ou fazer uma requisição (prompt do usuário) mais clara.


**P.S.**: É possível obrigar o modelo a sempre chamar alguma função, mas este é um recurso que vai ser útil mais raramente, acredito. Consulte a documentação do Google para ver como fazer isso.

## 4 - Entendendo Mais a Fundo

Para entender melhor o que acontece, vamos examinar o histórico de mensagens do primeiro chat (`chat1`).

Podemos ver o histórico do chat, usando a propriedade `.history`.

In [12]:
for content in chat1.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-------------')

user -> {'text': 'Quanto é 293874298 multiplicado por 7 milhões?'}
-------------
model -> {'function_call': {'name': 'multiply', 'args': {'a': 293874298.0, 'b': 7000000.0}}}
-------------
user -> {'function_response': {'name': 'multiply', 'response': {'result': 2057120086000000.0}}}
-------------
model -> {'text': '2.057120086e+15'}
-------------
user -> {'text': 'Tenho quatro mil e sete galpões, cada um com 1731 galinhas.\nPara cada galinha, preciso de 7.32 litro de água por dia.\nDe quantos litros de água eu preciso por dia?'}
-------------
model -> {'function_call': {'name': 'multiply', 'args': {'a': 4007.0, 'b': 1731.0}}}
-------------
user -> {'function_response': {'name': 'multiply', 'response': {'result': 6936117.0}}}
-------------
model -> {'function_call': {'name': 'multiply', 'args': {'a': 6936117.0, 'b': 7.32}}}
-------------
user -> {'function_response': {'name': 'multiply', 'response': {'result': 50772376.440000005}}}
-------------
model -> {'text': 'Você precisa de 50.772

### Entendo a Troca de Mensagens

Você notará que existem mensagens entre o "user" e o modelo ("model") que, na verdade, a biblioteca não retornou para nós! Elas foram trocadas internamente entre a classe `GenerativeModel` e o modelo remoto, na nuvem do Google. 

Por exemplo, nas quatro primeiras mensagens, vemos esta dinâmica:
1. O nosso prompt inicial perguntando sobre uma multiplicação é enviado ao modelo como um texto ('text') do usuário.
1. O modelo dá uma resposta com um conteúdo 'function_call' (e não 'text'). Esta é uma solicitação para chamar a função `multiply` passando os dois números que o usuário indicou.
1. Internamente, a classe `GenerativeModel` executa a função com os parâmetros indicado. Depois, atuando como 'user', ela envia uma resposta com um conteúdo 'function_response', que indica o retorno da função (o produto dos dois números).
1. O modelo, então, usa essa informação para gerar uma resposta com o conteúdo normal de texto ('text'). Esta é a mensagem que realmente a biblioteca "deixa passar" para nós.

Note, porém, que na pergunta sobre números primos, a resposta do modelo é diretamente do tipo 'text' -- pois não acontece (e não precisa de) nenhuma chamada de função.

### Tratando Chamadas de Função Diretamente

Caso você, como programador, queira tratar as chamadas de função diretamente, basta iniciar o chat com `model2.start_chat(enable_automatic_function_calling=False)`.

Assim, todas as mensagems de 'function_call' chegarão a você. Então, caberá a você, como programador, preparar o código que dispara a função solicitada e que envia, de volta, uma mensagem com a resposta ('function_response').

Pessoalmente, não vejo muito vantagem em fazer isso. Aliás, a chamada *automática* é um diferencial muito legal da biblioteca do Google. (No momento da escrita desta postagem, a biblioteca da OpenAI não oferece este recurso).

## 5. Conclusão

O recurso de **chamada de função** está presente em vários dos principals modelos de linguagem. 
É um recurso muito útil e poderoso para ampliar a capacidade dos modelos de linguagem.
Use sua criatividade para criar coisas legais com este recurso.

## Referências

- https://ai.google.dev/gemini-api/docs/get-started/python 
- https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python

Autor: Pablo A. Sampaio

03/jul/2024